# Семинар на тему Seq2Seq модели в машинном переводе

На лекции мы подробно познакомились с подходами к решению задачи машинного перевода.
Наиболее распространенными моделями последовательностей (seq2seq) являются модели кодер-декодер, которые (обычно) используют рекуррентную нейронную сеть (RNN) для кодирования исходного (входного) предложения в один вектор.  Вы можете думать о векторе контекста как об абстрактном представлении всего входного предложения. Этот вектор затем декодируется декодером, который учится выводить  предложение, генерируя его по одному слову за раз.

$h_t = \text{Encoder}(x_t, h_{t-1})$

У нас есть последовательность $X = \{x_1, x_2, ..., x_T\}$, где $x_1 = \text{<sos>;}, x_2 = \text{the}$, и так далее. Начальное состояние, $h_0$,  может быть инициализировано вектором из нулей или обучаемым.


Как только последнее слово, $x_T$, был подан на Encoder, мы  используем  информацию в  последнем скрытом состоянии, $h_T$, в зависимости от контекста вектор, т. е. $h_T $ это векторное представление всего исходного предложения.

После получения вектора всего предложения мы можем декодировать предложение уже на новом языке. На каждом шаге декодирования мы подаем правильное слово $y_t$,  дополняем это информацией о скрытом состоянии $s_{t-1}$, где  $s_t = \text{DecoderRNN}(y_t, s_{t-1})$


![alt text](https://i.stack.imgur.com/f6DQb.png)


Мы всегда используем $<sos>$ для первого входа в декодер, $y_1$, но для последующих входов, $y_{\text{from }t; 1}$, мы иногда будем использовать фактическое, основное истинное следующее слово в последовательности, $y_t$, а иногда использовать слово, предсказанное нашим декодером, $\hat{y}_{t-1}$. Использование настоящих токенов в декодере называется Teacher Forcing [можно тут посмотреть](https://machinelearningmastery.com/teacher-forcing-for-recurrent-neural-networks/)

Мы будем  использовать TorchText и spaCy( как токенизатор) , чтобы помочь вам выполнить всю необходимую предварительную обработку быстрее чем мы делали раньше. В данной работе вам предлагается написать модель Seq2Seq и обучить ее на Multi30k. В данном задание мы будем подавать на вход перевернутые предложения, так как авторы seq2seq считали, что это улучшает качество перевода.

In [0]:

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

import spacy

import random
import math
import time

from torchtext.datasets import TranslationDataset, Multi30k
from torchtext.data import Field, BucketIterator

In [0]:
seed = 43

random.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)
torch.backends.cudnn.deterministic = True

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [0]:
! python -m spacy download en
! python -m spacy download de


spacy_de = spacy.load('de')
spacy_en = spacy.load('en')


    Linking successful
    /usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/en

    You can now load the model via spacy.load('en')

    100% |████████████████████████████████| 38.2MB 134.3MB/s 
  Running setup.py install for de-core-news-sm ... done

    Linking successful
    /usr/local/lib/python3.6/dist-packages/de_core_news_sm -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/de

    You can now load the model via spacy.load('de')



In [0]:
def tokenize_de(text):
    """
    Tokenizes German text from a string into a list of strings (tokens) and reverses it
    """
    return [tok.text for tok in spacy_de.tokenizer(text)][::-1]

def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings (tokens)
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

# немецкий язык является полем SRC, а английский в поле TRG
SRC = Field(tokenize = tokenize_de, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

TRG = Field(tokenize = tokenize_en, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

In [0]:
# В датасете содержится ~ 30к предложений средняя длина которых 11
train_data, valid_data, test_data = Multi30k.splits(exts = ('.de', '.en'),  fields = (SRC, TRG))

downloading training.tar.gz


training.tar.gz: 100%|██████████| 1.21M/1.21M [00:01<00:00, 745kB/s] 


downloading validation.tar.gz


validation.tar.gz: 100%|██████████| 46.3k/46.3k [00:00<00:00, 229kB/s]


downloading mmt_task1_test2016.tar.gz


mmt_task1_test2016.tar.gz: 100%|██████████| 66.2k/66.2k [00:00<00:00, 218kB/s]


Давайте посмотрим что у нас с датасетом и сделаем словари для SRC и TGT

In [0]:
labels = ['train', 'validation', 'test']
dataloaders = [train_data, valid_data, test_data]
for d, l in zip(dataloaders, labels):
    print("Number of sentences in {} : {}".format(l, len(d.examples)))


Number of sentences in train : 29000
Number of sentences in validation : 1014
Number of sentences in test : 1000


In [0]:
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)
print("Number of words in source vocabulary", len(SRC.vocab))
print("Number of words in source vocabulary", len(TRG.vocab))

Number of words in source vocabulary 7853
Number of words in source vocabulary 5893


In [0]:
SRC.process("Ein klein gespenster")

tensor([[   2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,
            2,    2,    2,    2,    2,    2,    2,    2],
        [   0, 3833,    0,  664,    0,    0, 5884, 3833,    0,  664,    0, 5884,
            0,    0, 5884,    0,    0,    0, 5884, 7011],
        [   3,    3,    3,    3,    3,    3,    3,    3,    3,    3,    3,    3,
            3,    3,    3,    3,    3,    3,    3,    3]])

In [0]:
model(SRC.process("Ein klein gespenster"), np.array([]), teacher_forcing_ratio=0)

IndexError: ignored

## Encoder

Напишем для начала простой Encoder, который реализует следующий функционал:

$ (h_t, c_t) = \text{LSTM}(x_t, (h_{t-1}, c_{t-1}))$

В  методе forward мы передаем исходное предложение $X$, которое преобразуется в embeddings, к которым применяется dropout . Эти вектора затем передаются в RNN. Когда мы передадим всю последовательность RNN, он автоматически выполнит для нас рекуррентный расчет скрытых состояний по всей последовательности! Вы можете заметить, что мы не передаем начальное скрытое или состояние ячейки в RNN. Это происходит потому, что, как отмечено в документации, если никакое скрытое состояние/ячейка не передается RNN, он автоматически создаст начальное скрытое состояние/ячейка как тензор всех нулей.

In [0]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        """
        :param: input_dim is the size/dimensionality of the one-hot vectors that will be input to the encoder. This is equal to the input (source) vocabulary size.
        :param: emb_dim is the dimensionality of the embedding layer. This layer converts the one-hot vectors into dense vectors with emb_dim dimensions.
        :param: hid_dim is the dimensionality of the hidden and cell states.
        :param: n_layers is the number of layers in the RNN.
        :param: percentage of the dropout to use
        
        """
        super().__init__()
        
        self.input_dim = input_dim
        self.emb_dim = emb_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        self.dropout = dropout

        self.embedding = nn.Embedding(input_dim, emb_dim)
 
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        """
        :param: src sentences (src_len x batch_size)
        """
        # embedded = <TODO> (src_len x batch_size x embd_dim)
        embedded = self.embedding(src)
        # dropout over embedding
        embedded = self.dropout(embedded)
        outputs, (hidden, cell) = self.rnn(embedded)
        # [Attention return is for lstm, but you can also use gru]
        return hidden, cell

## Decoder
Похожий на Encoder, но со слоем проекцией, который переводит из hidden_dim в output

In [0]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()

        self.emb_dim = emb_dim
        self.hid_dim = hid_dim
        self.output_dim = output_dim
        self.n_layers = n_layers
        self.dropout = dropout
        
        self.embedding = nn.Embedding(self.output_dim, self.emb_dim)
        
        self.rnn = nn.LSTM(self.emb_dim, self.hid_dim, self.n_layers) #(lstm embd, hid, layers, dropout)
        
        self.out = nn.Linear(self.hid_dim, self.output_dim)# Projection :hid_dim x output_dim
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input_, hidden, cell):
        
        
        # (1x batch_size)
        input_ = input_.unsqueeze(0)
        
        # (1 x batch_size x emb_dim)
        embedded = self.embedding(input_)# embd over input and dropout 
        embedded = self.dropout(embedded)
                
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        
        #sent len and n directions will always be 1 in the decoder
        
        # (batch_size x output_dim)
        
        prediction = self.out(output.squeeze(0)) #project out of the rnn on the output dim 
        
        return prediction, hidden, cell


## Seq2Seq module

In [0]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        # Hidden dimensions of encoder and decoder must be equal
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        self._init_weights()  
    
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        """
        :param: src (src_len x batch_size)
        :param: tgt
        :param: teacher_forcing_ration : if 0.5 then every second token is the ground truth input
        """
        
        batch_size = trg.shape[1]
        max_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(max_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell = self.encoder(src) # TODO pass src throw encoder
        
        #first input to the decoder is the <sos> tokens
        input = trg[0, :] # TODO trg[idxs]
        
        for t in range(1, max_len):
            
            output, hidden, cell = self.decoder(input, hidden, cell) #TODO pass state and input throw decoder 
            outputs[t] = output
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.max(1)[1]
            input = (trg[t] if teacher_force else top1)
        
        return outputs
    
    def _init_weights(self):
        p = 0.08
        for name, param in self.named_parameters():
            nn.init.uniform_(param.data, -p, p)
        


In [0]:
input_dim = len(SRC.vocab)
output_dim = len(TRG.vocab)
src_embd_dim =  tgt_embd_dim = 256
hidden_dim = 512
num_layers =  2
dropout_prob = 0.2

batch_size = 128
PAD_IDX = TRG.vocab.stoi['<pad>']

iterators = BucketIterator.splits((train_data, valid_data, test_data),
                                  batch_size = batch_size, device = device)
train_iterator, valid_iterator, test_iterator = iterators

enc = Encoder(input_dim, src_embd_dim, hidden_dim, num_layers, dropout_prob)
dec = Decoder(output_dim, tgt_embd_dim, hidden_dim, num_layers, dropout_prob)
model = Seq2Seq(enc, dec, device).to(device)


In [0]:
model

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(7853, 256)
    (rnn): LSTM(256, 512, num_layers=2)
    (dropout): Dropout(p=0.2)
  )
  (decoder): Decoder(
    (embedding): Embedding(5893, 256)
    (rnn): LSTM(256, 512, num_layers=2)
    (out): Linear(in_features=512, out_features=5893, bias=True)
    (dropout): Dropout(p=0.2)
  )
)

In [0]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        output = output[1:].view(-1, output.shape[-1])
        trg = trg[1:].view(-1)
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [0]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.src
            trg = batch.trg

            output = model(src, trg, 0) #turn off teacher forcing !!

            output = output[1:].view(-1, output.shape[-1])
            trg = trg[1:].view(-1)


            loss = criterion(output, trg)
            
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [0]:
max_epochs = 10
CLIP = 1

# TODO
optimizer = optim.Adam(model.parameters(), lr = 0.001)
criterion = nn.CrossEntropyLoss(ignore_index = PAD_IDX)

best_valid_loss = float('inf')

for epoch in range(max_epochs):
    
    
    train_loss = round(train(model, train_iterator, optimizer, criterion, CLIP), 5)
    valid_loss = round(evaluate(model, valid_iterator, criterion),5)
    
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'model.pt')
    
    print('Epoch: {} \n Train Loss {}  Val loss {}:'.format(epoch, train_loss, valid_loss))
    print('Train Perplexity {}  Val Perplexity {}:'.format(np.exp(train_loss), np.exp(valid_loss)))


Epoch: 0 
 Train Loss 4.26041  Val loss 5.10416:
Train Perplexity 70.83902149988542  Val Perplexity 164.70565965310226:
Epoch: 1 
 Train Loss 4.22852  Val loss 5.07115:
Train Perplexity 68.61560589222772  Val Perplexity 159.35748311412488:
Epoch: 2 
 Train Loss 4.23805  Val loss 5.0519:
Train Perplexity 69.27263840361465  Val Perplexity 156.31918896791194:
Epoch: 3 
 Train Loss 4.18014  Val loss 5.06719:
Train Perplexity 65.37500507407512  Val Perplexity 158.72767532345105:
Epoch: 4 
 Train Loss 4.12467  Val loss 4.98157:
Train Perplexity 61.847396241646955  Val Perplexity 145.702955816573:
Epoch: 5 
 Train Loss 4.08609  Val loss 4.91129:
Train Perplexity 59.506764787709216  Val Perplexity 135.81450216037183:
Epoch: 6 
 Train Loss 4.04193  Val loss 4.87235:
Train Perplexity 56.93612355142656  Val Perplexity 130.6275311818444:
Epoch: 7 
 Train Loss 4.03445  Val loss 4.87374:
Train Perplexity 56.511830182740894  Val Perplexity 130.80922970140324:
Epoch: 8 
 Train Loss 3.96459  Val loss 4

In [0]:
test_loss = evaluate(model, test_iterator, criterion)

print('| Test Loss: {} Test PPL:{}|'.format(test_loss, np.exp(test_loss)))

| Test Loss: 4.754251182079315 Test PPL:116.07670029917236|
